Create bash scripts

In [10]:
import os
from pathlib import Path

from light_by_light.vacem_ini import W_to_E0
from light_by_light.utils import write_yaml

In [17]:
template = '''#!/bin/bash
#SBATCH --job-name={jobname}
#SBATCH --partition=hij-gpu
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --time=1-00:00:00
#SBATCH --mem=200GB
#SBATCH --output=/home/wi73yus/bash_output/{jobname}.%j.log
#SBATCH --error=/home/wi73yus/bash_output/{jobname}.%j.err

source ~/.bashrc  #  not automatically read by slurm
conda activate lbl  # activate here the anaconda environment you want to use

python {script_path} {script_kwargs}
'''

### Bash script for 1d grid scan

#### Create yaml scripts

In [13]:
# Laser pulse parameters
laser_params = []
for i in range(2):
    laser = {
        'tau': 25e-15,
        'lam': 800e-9,
        'w0': 2*800e-9,
        'W': 25,
        'theta': 180*i,
        'phi': 0,
        'beta': 0,
        'solution': 'gaussian_paraxial'
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 1,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'gridscan'
geometry = 'xz'
low_memory_mode = False
n_threads = 24
pol_idx = 0

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx
}

# Variable parameters 
laser_key = 'laser_0'
param_key = 'tau'
grid = [20,40,3,1e-15]
vary_params = {
    'lasers': {laser_key: {param_key: grid}}
}

# Save yaml files
save_path = 'runs/test/gridscan/experiment/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}vary_params.yml'
write_yaml(vary_yaml, vary_params)

In [15]:
jobname = 'test_gridscan'
global_path = '/home/wi73yus/light-by-light'
script_path = f'{global_path}/cluster/gridscan_cluster.py'
global_save_path = f'{global_path}/{save_path}'
script_kwargs = f'--save_path={global_save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'/home/wi73yus/light-by-light/bash_scripts/{jobname}.slurm'
with open(bash_path, 'w+') as f:
    f.write(bash)

### Bash script for optuna optimization

In [18]:
# Laser pulse parameters
laser_params = []
for i in range(2):
    laser = {
        'tau': 25e-15,
        'lam': 800e-9,
        'w0': 2*800e-9,
        'W': 25,
        'theta': 180*i,
        'phi': 0,
        'beta': 0,
        'solution': 'gaussian_paraxial'
    }
    laser['E0'] = float(W_to_E0(laser))
    laser_params.append(laser)

laser_params = {f'laser_{i}': value for i,value in enumerate(laser_params)}

# Simulation box parameters
box_factors = {
    'long': 8,
    'trans': 20,
    't': 2
}

resolutions = {
    'spatial': 1,
    't': 1
}
simbox_params = {'box_factors': box_factors,
                 'resolutions': resolutions}

# Other parameters
mode = 'optuna'
geometry = 'xz'
low_memory_mode = False
n_threads = 24
pol_idx = 0

# Optuna params
obj_param = 'N_total'
n_trials = 10

data = {
    '_mode': mode,
    'lasers': laser_params,
    'simbox_params': simbox_params,
    'geometry': geometry,
    'low_memory_mode': low_memory_mode,
    'n_threads': n_threads,
    'pol_idx': pol_idx,
    'obj_param': obj_param,
    'n_trials': n_trials
}

# Variable parameters 
laser_key = 'laser_0'
param_key = 'tau'
grid = [25,40,1e-15,'uniform']
vary_params = {laser_key:
    {param_key: grid}
}


# Save yaml files
save_path = 'runs/test/optuna/experiment/'
Path(os.path.dirname(save_path)).mkdir(parents=True, exist_ok=True)

default_yaml = f'{save_path}default_params.yml'
write_yaml(default_yaml, data)

vary_yaml = f'{save_path}optuna_params.yml'
write_yaml(vary_yaml, vary_params)

In [19]:
jobname = 'test_optuna'
global_path = '/home/wi73yus/light-by-light'
script_path = f'{global_path}/cluster/optuna_cluster.py'
global_save_path = f'{global_path}/{save_path}'
script_kwargs = f'--save_path={global_save_path}'

bash = template.format(jobname=jobname,
                       n_cpus=n_threads,
                       script_path=script_path,
                       script_kwargs=script_kwargs)

bash_path = f'/home/wi73yus/light-by-light/bash_scripts/{jobname}.slurm'
with open(bash_path, 'w+') as f:
    f.write(bash)